In [1]:
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os
import json

c:\Users\walea\miniconda3\envs\fitness\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Create a Pinecone index
pc.create_index(
    name="fitness",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [3]:
# Load the review data
data = json.load(open("fitness.json"))

In [4]:
processed_data = []
client = OpenAI()

# Create embeddings for each exercise
for exercise in data["fitness"]:
    response = client.embeddings.create(
        input=exercise['instructions'], model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": exercise["exercise_name"],
            "metadata": {
                "type_of_activity": exercise["type_of_activity"],
                "type_of_equipment": exercise["type_of_equipment"],
                "body_part": exercise["body_part"],
                "type": exercise["type"],
                "muscle_groups_activated": exercise["muscle_groups_activated"],
                "instructions": exercise["instructions"]
            }
        }
    )

In [5]:
# Insert the embeddings into the Pinecone index
index = pc.Index("fitness")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)
print(f"Upserted count: {upsert_response['upserted_count']}")

Upserted count: 207


In [6]:
# Print index statistics
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 207}},
 'total_vector_count': 207}
